In [1]:
import pandas as pd
import numpy as np
import miceforest as mf


In [3]:
df = pd.read_csv("../data/V2_TEST_DATA_WITHDEMOGRAPHICS.csv")

In [5]:
df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])

In [6]:
df.head()

,CrimeDate,Description,District,Neighborhood,Premise,Month,Hour,Outside,Weapon_FIREARM,Weapon_HANDS,...,perc18_24,perc25_64,perc65up,perc_asian,perc_aa,perc_hisp,perc_white,median_price_homes_sold,racial_diversity_index,num_households
0,2018-12-30,7,3,HOMELAND,78,12,0,1,0,0,...,25.103793,43.962919,15.253370,10.805892,15.947222,3.469260,64.852414,338350.0,54.141457,347500.0
1,2018-12-29,4,0,UPTON,69,12,0,0,0,0,...,10.168599,45.584826,12.465753,1.401475,90.769231,1.064278,4.573235,105500.0,18.546981,53465.0
2,2018-12-29,12,2,PULASKI INDUSTRIAL AREA,38,12,0,0,0,0,...,5.425101,50.024291,11.303644,1.943320,31.433198,26.542510,38.801619,125000.0,77.658458,83000.0
3,2018-12-28,4,6,RIVERSIDE,50,12,0,0,0,0,...,4.993910,77.425903,6.645013,2.287184,2.354852,3.315740,90.634727,350000.0,20.090743,306000.0
4,2018-12-28,3,4,LANGSTON HUGHES,78,12,0,1,0,0,...,9.433962,49.834663,24.129547,0.437658,94.728652,0.816962,2.655125,69680.0,11.410513,40000.0


In [7]:
df.columns


Index(['CrimeDate', 'Description', 'District', 'Neighborhood', 'Premise',
       'Month', 'Hour', 'Outside', 'Weapon_FIREARM', 'Weapon_HANDS',
       'Weapon_KNIFE', 'Weapon_NONE', 'Weapon_OTHER', 'Holiday', 'Weekend',
       'CSA', 'Year', 'median_household_income', 'households_below_poverty',
       'perc18_24', 'perc25_64', 'perc65up', 'perc_asian', 'perc_aa',
       'perc_hisp', 'perc_white', 'median_price_homes_sold',
       'racial_diversity_index', 'num_households'],
      dtype='object')

In [8]:
df = df.drop(columns=["CrimeDate","CSA"])


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51730 entries, 0 to 51729
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Description               51730 non-null  int64  
 1   District                  51730 non-null  int64  
 2   Neighborhood              51730 non-null  object 
 3   Premise                   51730 non-null  int64  
 4   Month                     51730 non-null  int64  
 5   Hour                      51730 non-null  int64  
 6   Outside                   51730 non-null  int64  
 7   Weapon_FIREARM            51730 non-null  int64  
 8   Weapon_HANDS              51730 non-null  int64  
 9   Weapon_KNIFE              51730 non-null  int64  
 10  Weapon_NONE               51730 non-null  int64  
 11  Weapon_OTHER              51730 non-null  int64  
 12  Holiday                   51730 non-null  int64  
 13  Weekend                   51730 non-null  int64  
 14  Year  

Wonder if this will work with the saved imputer...


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
Neighborhood = pd.read_csv("../data/V2_Neighborhood_decoder.csv", header=None)
le.fit(Neighborhood[0])

LabelEncoder()

In [12]:
df["Neighborhood"]=le.transform(df["Neighborhood"])

In [15]:
import pickle

In [16]:
impute = pickle.load(open("MICE_MODEL.sav", "rb"))


In [19]:
completed_data = impute.impute_new_data(new_data=df)

ValueError: Columns are not the same as kernel data

In [20]:
len(df)

51730